In [1138]:
import pandas as pd
import numpy as np

import itertools
import requests
import mysql.connector
import config
from bs4 import BeautifulSoup 
import json
from functools import reduce
from pandas import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 310)


# Data import/Manipulation from CSV file

In [452]:
# from kaggle this is a dataset of MyAnimeList.net of popular anime

# exporting csv file to a DataFrame only using columns listed
data = pd.read_csv("Video_Games_Sales_as_at_22_Dec_2016.csv", usecols=['Name','Platform','Year_of_Release','Genre','Publisher','NA_Sales', 'EU_Sales', 'JP_Sales','Other_Sales','Global_Sales','Critic_Score', 'User_Score','User_Count','Developer','Rating'])
# 
# dropping all null values
data.dropna(inplace = True)
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6644 entries, 180 to 16141
Data columns (total 15 columns):
Name               6644 non-null object
Platform           6644 non-null object
Year_of_Release    6644 non-null float64
Genre              6644 non-null object
Publisher          6644 non-null object
NA_Sales           6644 non-null float64
EU_Sales           6644 non-null float64
JP_Sales           6644 non-null float64
Other_Sales        6644 non-null float64
Global_Sales       6644 non-null float64
Critic_Score       6644 non-null float64
User_Score         6644 non-null object
User_Count         6644 non-null float64
Developer          6644 non-null object
Rating             6644 non-null object
dtypes: float64(8), object(7)
memory usage: 830.5+ KB


In [1101]:
data2=data.copy()
data2.drop('User_Count', axis =1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,Developer,Rating
180,Monster Hunter Stories,3DS,2017.0,Action,Capcom,0.00,0.00,0.28,0.00,0.28,79.0,8.2,Capcom,E10+
181,The Elder Scrolls V: Skyrim,PS4,2016.0,Role-Playing,Bethesda Softworks,0.48,0.59,0.04,0.21,1.33,81.0,7.1,Bethesda Game Studios,M
182,Rise of the Tomb Raider,PS4,2016.0,Adventure,Square Enix,0.23,0.53,0.04,0.14,0.94,88.0,8.1,"Crystal Dynamics, Nixxes Software",M
183,MLB 16: The Show,PS4,2016.0,Action,Sony Computer Entertainment,0.72,0.00,0.00,0.17,0.89,85.0,7.3,SCEA San Diego Studios,E
184,Doom (2016),XOne,2016.0,Shooter,Bethesda Softworks,0.44,0.25,0.00,0.07,0.76,87.0,8.4,id Software,M
185,The Elder Scrolls V: Skyrim,XOne,2016.0,Role-Playing,Bethesda Softworks,0.41,0.27,0.00,0.07,0.74,82.0,7.5,Bethesda Game Studios,M
186,Minecraft,WiiU,2016.0,Misc,Microsoft Game Studios,0.28,0.17,0.18,0.04,0.68,77.0,7.3,4J Studios,E10+
189,Mario & Sonic at the Rio 2016 Olympic Games,3DS,2016.0,Action,Nintendo,0.05,0.21,0.25,0.02,0.53,60.0,6.8,"Nintendo, Sega Sports R&D",E10+
190,Hyrule Warriors,3DS,2016.0,Action,Nintendo,0.17,0.13,0.11,0.03,0.43,70.0,7.4,Omega Force,T
192,F1 2016 (Codemasters),PS4,2016.0,Racing,Codemasters,0.03,0.29,0.01,0.05,0.39,82.0,7.9,Codemasters,E


In [496]:
# copy dataframe to new DF, and converting FLOAT's to INT's
data_le = data.copy()

# makes Year_of_Release into INT it was a FLOAT and manually scaling 
data_le['Year_of_Release']=data_le['Year_of_Release'].astype(int)
data_le['Critic_Score']=data_le['Critic_Score'].astype(int)
data_le['User_Count']=data_le['User_Count'].astype(float)
data_le['User_Score']=data_le['User_Score'].astype(float) * 10
data_le['NA_Sales']=data_le['NA_Sales'].astype(float)
data_le['EU_Sales']=data_le['EU_Sales'].astype(float)
data_le['JP_Sales']=data_le['JP_Sales'].astype(float)
data_le['Other_Sales']=data_le['Other_Sales'].astype(float)
data_le['Global_Sales']=data_le['Global_Sales'].astype(float)

data_le_nums = data_le.drop(columns=['Name','Platform','Genre','Publisher', 'Developer', 'Rating'])

data_le = data_le.sort_values(by=['Name'])

data_original=data_le.copy()


In [497]:
data_original.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,User_Count,Developer,Rating
180,Monster Hunter Stories,3DS,2017,Action,Capcom,0.00,0.00,0.28,0.00,0.28,79,82.0,91.0,Capcom,E10+
181,The Elder Scrolls V: Skyrim,PS4,2016,Role-Playing,Bethesda Softworks,0.48,0.59,0.04,0.21,1.33,81,71.0,624.0,Bethesda Game Studios,M
182,Rise of the Tomb Raider,PS4,2016,Adventure,Square Enix,0.23,0.53,0.04,0.14,0.94,88,81.0,1082.0,"Crystal Dynamics, Nixxes Software",M
183,MLB 16: The Show,PS4,2016,Action,Sony Computer Entertainment,0.72,0.00,0.00,0.17,0.89,85,73.0,182.0,SCEA San Diego Studios,E
184,Doom (2016),XOne,2016,Shooter,Bethesda Softworks,0.44,0.25,0.00,0.07,0.76,87,84.0,679.0,id Software,M


## Api Calls for the Console info

In [58]:
api_key = config.api_key

headers = {
    'x-rapidapi-host': "rawg-video-games-database.p.rapidapi.com",
    'x-rapidapi-key': f"{api_key}"
    }
console_data = []
for i in range(1,100):    ### Consoles in api call some are empty and i know there isn't more then 100 consoles
    url = f"https://rawg-video-games-database.p.rapidapi.com/platforms/{i}"
    response = requests.request("GET", url, headers=headers)
    if response == '{"detail":"Not found."}':
        break
    else:
        console_data.append(response.json())

console_data

[{'id': 1,
  'name': 'Xbox One',
  'slug': 'xbox-one',
  'games_count': 2942,
  'image_background': 'https://media.rawg.io/media/games/9fa/9fa63622543e5d4f6d99aa9d73b043de.jpg',
  'description': '<p>Xbox One is a home video game console released by Microsoft in 2013. Unlike its main competitor — PlayStation 4, the accent in the development was made in multitasking and using a console as a home media player to watch TV, listen to music and play the games. There is three hardware option available to gamers: original console, One S version (with a smaller body, no external power supply, and native HDR/4k support) and One X (pro-gamer-oriented system with updated hardware and 4k rendering in real time). While PS4 is known for exclusive-orientation, Xbox One&#39;s games are mostly multi-platform titles. This supports Microsoft&#39;s initial idea to prevent selling used games between the players. However, given the competition from the Sony side, the corporation quickly gave up. Among the no

In [704]:
#making copy of api info
console_data_copy = console_data.copy()
console_data_copy

[{'id': 1,
  'name': 'Xbox One',
  'slug': 'xbox-one',
  'games_count': 2942,
  'image_background': 'https://media.rawg.io/media/games/9fa/9fa63622543e5d4f6d99aa9d73b043de.jpg',
  'description': '<p>Xbox One is a home video game console released by Microsoft in 2013. Unlike its main competitor — PlayStation 4, the accent in the development was made in multitasking and using a console as a home media player to watch TV, listen to music and play the games. There is three hardware option available to gamers: original console, One S version (with a smaller body, no external power supply, and native HDR/4k support) and One X (pro-gamer-oriented system with updated hardware and 4k rendering in real time). While PS4 is known for exclusive-orientation, Xbox One&#39;s games are mostly multi-platform titles. This supports Microsoft&#39;s initial idea to prevent selling used games between the players. However, given the competition from the Sony side, the corporation quickly gave up. Among the no

In [59]:
# removing empty "{"detail":"Not found."}" from api call list of dictionaries
console_data_copy = [x for x in console_data_copy if keys is not 'detail' in x]

console_data_copy

NameError: name 'keys' is not defined

In [678]:
# for i in range(0,len(console_data_copy)):
#     if console_data_copy[i] == {'detail': 'Not found.'}:
#         console_data_copy.remove(console_data_copy[i])
#         continue
#     else:
#         break
# console_data_copy



[{'id': 1,
  'name': 'Xbox One',
  'slug': 'xbox-one',
  'games_count': 2942,
  'image_background': 'https://media.rawg.io/media/games/9fa/9fa63622543e5d4f6d99aa9d73b043de.jpg',
  'description': '<p>Xbox One is a home video game console released by Microsoft in 2013. Unlike its main competitor — PlayStation 4, the accent in the development was made in multitasking and using a console as a home media player to watch TV, listen to music and play the games. There is three hardware option available to gamers: original console, One S version (with a smaller body, no external power supply, and native HDR/4k support) and One X (pro-gamer-oriented system with updated hardware and 4k rendering in real time). While PS4 is known for exclusive-orientation, Xbox One&#39;s games are mostly multi-platform titles. This supports Microsoft&#39;s initial idea to prevent selling used games between the players. However, given the competition from the Sony side, the corporation quickly gave up. Among the no

In [672]:
# for i in range(0,len(console_data_copy)):
#     data_consoles = pd.DataFrame.from_dict(console_data_copy[i])

# data_consoles

ValueError: If using all scalar values, you must pass an index

In [742]:
data_consoles = pd.DataFrame.from_dict(console_data_copy)
data_consoles.drop(columns = ['year_end' ,'year_start', 'detail', 'image'])


,description,games_count,id,image_background,name,slug
0,<p>Xbox One is a home video game console relea...,2942.0,1.0,https://media.rawg.io/media/games/9fa/9fa63622...,Xbox One,xbox-one
1,NaN,NaN,NaN,NaN,NaN,NaN
2,<p>iOS is an operating system initially develo...,64520.0,3.0,https://media.rawg.io/media/screenshots/10f/10...,iOS,ios
3,"<p>PC games, or personal computer games, start...",194331.0,4.0,https://media.rawg.io/media/games/d1a/d1a2e99a...,PC,pc
4,<p>MacOS (previously OS X) is an operating sys...,45674.0,5.0,https://media.rawg.io/media/games/b45/b45575f3...,macOS,macos
5,<p>Linux is a free and open-source software op...,30416.0,6.0,https://media.rawg.io/media/games/e04/e04963f3...,Linux,linux
6,<p>Nintendo Switch is a hybrid video game cons...,3025.0,7.0,https://media.rawg.io/media/games/2ad/2add0a45...,Nintendo Switch,nintendo-switch
7,<p>Nintendo 3DS is Nintendo’s fifth major port...,1706.0,8.0,https://media.rawg.io/media/screenshots/601/60...,Nintendo 3DS,nintendo-3ds
8,<p>Nintendo DS is Nintendo’s fourth major port...,2239.0,9.0,https://media.rawg.io/media/screenshots/d4d/d4...,Nintendo DS,nintendo-ds
9,<p>Nintendo Wii U is the sixth major home vide...,1271.0,10.0,https://media.rawg.io/media/games/b4e/b4e4c73d...,Wii U,wii-u


In [732]:
data_consoles.dropna(how='all')

,description,detail,games_count,id,image,image_background,name,slug,year_end,year_start


In [748]:
data_consoles2 = data_consoles
data_consoles2.usecols=['id' ,'year_start', 'detail', 'image']

data_consoles2 = data_consoles2.dropna(how='all')


In [749]:
data_consoles2

,description,detail,games_count,id,image,image_background,name,slug,year_end,year_start
0,<p>Xbox One is a home video game console relea...,NaN,2942.0,1.0,NaN,https://media.rawg.io/media/games/9fa/9fa63622...,Xbox One,xbox-one,NaN,NaN
1,NaN,Not found.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<p>iOS is an operating system initially develo...,NaN,64520.0,3.0,NaN,https://media.rawg.io/media/screenshots/10f/10...,iOS,ios,NaN,NaN
3,"<p>PC games, or personal computer games, start...",NaN,194331.0,4.0,NaN,https://media.rawg.io/media/games/d1a/d1a2e99a...,PC,pc,NaN,NaN
4,<p>MacOS (previously OS X) is an operating sys...,NaN,45674.0,5.0,NaN,https://media.rawg.io/media/games/b45/b45575f3...,macOS,macos,NaN,NaN
5,<p>Linux is a free and open-source software op...,NaN,30416.0,6.0,NaN,https://media.rawg.io/media/games/e04/e04963f3...,Linux,linux,NaN,NaN
6,<p>Nintendo Switch is a hybrid video game cons...,NaN,3025.0,7.0,NaN,https://media.rawg.io/media/games/2ad/2add0a45...,Nintendo Switch,nintendo-switch,NaN,NaN
7,<p>Nintendo 3DS is Nintendo’s fifth major port...,NaN,1706.0,8.0,NaN,https://media.rawg.io/media/screenshots/601/60...,Nintendo 3DS,nintendo-3ds,NaN,NaN
8,<p>Nintendo DS is Nintendo’s fourth major port...,NaN,2239.0,9.0,NaN,https://media.rawg.io/media/screenshots/d4d/d4...,Nintendo DS,nintendo-ds,NaN,NaN
9,<p>Nintendo Wii U is the sixth major home vide...,NaN,1271.0,10.0,NaN,https://media.rawg.io/media/games/b4e/b4e4c73d...,Wii U,wii-u,NaN,NaN


## Web scrapping console sell 

In [245]:
# Scrapping console sales data
URL = 'http://www.vgchartz.com/analysis/platform_totals/Hardware/Global/'
r = requests.get(URL)


# Combining both urls into one 'soup'
soup = BeautifulSoup(r.content , 'html.parser')

#printing in sytlized text to read easier
print(soup.prettify()) 


<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- VGCHARTZ MAIN HEADER TAGS -->
  <!-- Venatus Market Ad-Manager (vgchartz.com) -->
  <script>
   (function(){document.write('<div id="vmv3-ad-manager" style="display:none"></div>');document.getElementById("vmv3-ad-manager").innerHTML='<iframe id="vmv3-frm" src="javascript:\'<html><body></body></html>\'" width="0" height="0" data-mode="scan" data-site-id="5b11330346e0fb00017cd841"></iframe>';var a=document.getElementById("vmv3-frm");a=a.contentWindow?a.contentWindow:a.contentDocument;a.document.open();a.document.write('<script src="https://hb.vntsm.com/v3/live/ad-manager.min.js" type="text/javascript" async>'+'</scr'+'ipt>');a.document.close()})();
  </script>
  <!-- / Venatus Market Ad-Manager (vgchartz.com) -->
  <meta content="4J-ohFOExhky4T8zKx5Nz04hdjbaxo52B6qVrLlM8o0" name="google-site-verification"/>
  <script src="http://www.google-analytics.com/urchin.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   _uac

In [385]:
list(soup.children)

['html', '\n', <html lang="en">
 <head>
 <!-- VGCHARTZ MAIN HEADER TAGS -->
 <!-- Venatus Market Ad-Manager (vgchartz.com) -->
 <script>
     (function(){document.write('<div id="vmv3-ad-manager" style="display:none"></div>');document.getElementById("vmv3-ad-manager").innerHTML='<iframe id="vmv3-frm" src="javascript:\'<html><body></body></html>\'" width="0" height="0" data-mode="scan" data-site-id="5b11330346e0fb00017cd841"></iframe>';var a=document.getElementById("vmv3-frm");a=a.contentWindow?a.contentWindow:a.contentDocument;a.document.open();a.document.write('<script src="https://hb.vntsm.com/v3/live/ad-manager.min.js" type="text/javascript" async>'+'</scr'+'ipt>');a.document.close()})();
     </script>
 <!-- / Venatus Market Ad-Manager (vgchartz.com) -->
 <meta content="4J-ohFOExhky4T8zKx5Nz04hdjbaxo52B6qVrLlM8o0" name="google-site-verification"/>
 <script src="http://www.google-analytics.com/urchin.js" type="text/javascript"></script>
 <script type="text/javascript">
         _uac

In [386]:
[type(item) for item in list(soup.children)]


[bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [388]:
html = list(soup.children)[2]
html

<html lang="en">
<head>
<!-- VGCHARTZ MAIN HEADER TAGS -->
<!-- Venatus Market Ad-Manager (vgchartz.com) -->
<script>
    (function(){document.write('<div id="vmv3-ad-manager" style="display:none"></div>');document.getElementById("vmv3-ad-manager").innerHTML='<iframe id="vmv3-frm" src="javascript:\'<html><body></body></html>\'" width="0" height="0" data-mode="scan" data-site-id="5b11330346e0fb00017cd841"></iframe>';var a=document.getElementById("vmv3-frm");a=a.contentWindow?a.contentWindow:a.contentDocument;a.document.open();a.document.write('<script src="https://hb.vntsm.com/v3/live/ad-manager.min.js" type="text/javascript" async>'+'</scr'+'ipt>');a.document.close()})();
    </script>
<!-- / Venatus Market Ad-Manager (vgchartz.com) -->
<meta content="4J-ohFOExhky4T8zKx5Nz04hdjbaxo52B6qVrLlM8o0" name="google-site-verification"/>
<script src="http://www.google-analytics.com/urchin.js" type="text/javascript"></script>
<script type="text/javascript">
        _uacct = "UA-2243248-1";
     

In [390]:
list(html.children)


['\n', <head>
 <!-- VGCHARTZ MAIN HEADER TAGS -->
 <!-- Venatus Market Ad-Manager (vgchartz.com) -->
 <script>
     (function(){document.write('<div id="vmv3-ad-manager" style="display:none"></div>');document.getElementById("vmv3-ad-manager").innerHTML='<iframe id="vmv3-frm" src="javascript:\'<html><body></body></html>\'" width="0" height="0" data-mode="scan" data-site-id="5b11330346e0fb00017cd841"></iframe>';var a=document.getElementById("vmv3-frm");a=a.contentWindow?a.contentWindow:a.contentDocument;a.document.open();a.document.write('<script src="https://hb.vntsm.com/v3/live/ad-manager.min.js" type="text/javascript" async>'+'</scr'+'ipt>');a.document.close()})();
     </script>
 <!-- / Venatus Market Ad-Manager (vgchartz.com) -->
 <meta content="4J-ohFOExhky4T8zKx5Nz04hdjbaxo52B6qVrLlM8o0" name="google-site-verification"/>
 <script src="http://www.google-analytics.com/urchin.js" type="text/javascript"></script>
 <script type="text/javascript">
         _uacct = "UA-2243248-1";
     

In [389]:
# 
console_sales_table = soup.find_all(class_='chart')
console_sales_table


[<table cellpadding="0" cellspacing="0" class="chart" id="myTable" width="100%">
 <thead>
 <tr>
 <th>Pos</th>
 <th style="text-align:left;">Platform</th>
 <th><center><a href="http://www.vgchartz.com/analysis/platform_totals/Hardware/NA/">North America</a></center></th>
 <th><center><a href="http://www.vgchartz.com/analysis/platform_totals/Hardware/EU/">Europe</a></center></th>
 <th><center><a href="http://www.vgchartz.com/analysis/platform_totals/Hardware/JP/">Japan</a></center></th>
 <th><center><a href="http://www.vgchartz.com/analysis/platform_totals/Hardware/ROW/">Rest of World</a></center></th>
 <th><center><a href="http://www.vgchartz.com/analysis/platform_totals/Hardware/Global/" id="selected">Global</a></center></th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>1</td>
 <td><a href="http://www.vgchartz.com/gamedb/games.php?console=PS2">PlayStation 2 (PS2)</a></td>
 <td><center>53.65</center></td>
 <td><center>55.28</center></td>
 <td><center>23.18</center></td>
 <td><center>25.57</cent

In [400]:
table_header=soup.find_all('th').get_text()

AttributeError: ResultSet object has no attribute 'get_text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [399]:
table_data=soup.find_all('td')
table_data

AttributeError: ResultSet object has no attribute 'get_text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [14]:
data

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,6.6,129.0,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,8.4,594.0,Nintendo,E
11,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.71,7.47,4.13,1.90,23.21,91.0,8.6,464.0,Nintendo,E
13,Wii Fit,Wii,2007.0,Sports,Nintendo,8.92,8.03,3.60,2.15,22.70,80.0,7.7,146.0,Nintendo,E
14,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,15.00,4.89,0.24,1.69,21.81,61.0,6.3,106.0,Good Science Studio,E
15,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.01,8.49,2.53,1.77,21.79,80.0,7.4,52.0,Nintendo,E


## Merging all console types per game into one row

In [498]:
df = data_le.groupby(['Name']).agg({'Platform': ','.join},{'Genre': ','.join},{'Developer': ','.join})
df.head()
# data_name = data_le.groupby('Name')['Platform'].apply(',' .join).reset_index()


,Platform
Name,
.hack//Infection Part 1,PS2
.hack//Mutation Part 2,PS2
.hack//Outbreak Part 3,PS2
007 Racing,PS
007: Quantum of Solace,"Wii,X360,PS3,PC"
007: The World is not Enough,PS
1 vs. 100,DS
1701 A.D.,PC
18 Wheeler: American Pro Trucker,"GC,PS2"


In [499]:
data_le = data_le.merge(df, how='inner', left_on='Name', right_on='Name')

In [500]:
data_le = data_le.drop_duplicates(subset='Name', keep='first', inplace=False)


In [501]:
data_le['Platform']=data_le['Platform_y']

In [502]:
data_le = data_le.drop(columns=['Platform_x','Platform_y'])

In [503]:
data_le.rename(columns={'Platform_y':'Platform'}, inplace=True)

In [505]:
data_le.head()

,Name,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,User_Count,Developer,Rating,Platform
0,Monster Hunter Stories,2017,Action,Capcom,0.00,0.00,0.28,0.00,0.28,79,82.0,91.0,Capcom,E10+,3DS
1,The Elder Scrolls V: Skyrim,2016,Role-Playing,Bethesda Softworks,0.48,0.59,0.04,0.21,1.33,81,71.0,624.0,Bethesda Game Studios,M,"PS4,XOne,X360,PC,PS3"
6,Rise of the Tomb Raider,2016,Adventure,Square Enix,0.23,0.53,0.04,0.14,0.94,88,81.0,1082.0,"Crystal Dynamics, Nixxes Software",M,"PS4,XOne"
8,MLB 16: The Show,2016,Action,Sony Computer Entertainment,0.72,0.00,0.00,0.17,0.89,85,73.0,182.0,SCEA San Diego Studios,E,PS4
9,Doom (2016),2016,Shooter,Bethesda Softworks,0.44,0.25,0.00,0.07,0.76,87,84.0,679.0,id Software,M,"XOne,PC"


In [506]:
#joinning all games with all consoles the game under 1 column

data_name = data_le.groupby('Name')['Platform'].apply(',' .join).reset_index()
data_name2 = data_le.groupby(['Name'], as_index = False).agg({'Platform': ','.join})


data_name.head(100)


dataframe=[data_le,data_name2]
data_with_platforms =  reduce(lambda left,right: pd.merge(left, right, how='inner' ,left_index=True, right_index=True),  dataframe)
data_with_platforms

,Name_x,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,User_Count,Developer,Rating,Platform_x,Name_y,Platform_y
0,Monster Hunter Stories,2017,Action,Capcom,0.00,0.00,0.28,0.00,0.28,79,82.0,91.0,Capcom,E10+,3DS,.hack//Infection Part 1,PS2
1,The Elder Scrolls V: Skyrim,2016,Role-Playing,Bethesda Softworks,0.48,0.59,0.04,0.21,1.33,81,71.0,624.0,Bethesda Game Studios,M,"PS4,XOne,X360,PC,PS3",.hack//Mutation Part 2,PS2
6,Rise of the Tomb Raider,2016,Adventure,Square Enix,0.23,0.53,0.04,0.14,0.94,88,81.0,1082.0,"Crystal Dynamics, Nixxes Software",M,"PS4,XOne",1 vs. 100,DS
8,MLB 16: The Show,2016,Action,Sony Computer Entertainment,0.72,0.00,0.00,0.17,0.89,85,73.0,182.0,SCEA San Diego Studios,E,PS4,18 Wheeler: American Pro Trucker,"GC,PS2"
9,Doom (2016),2016,Shooter,Bethesda Softworks,0.44,0.25,0.00,0.07,0.76,87,84.0,679.0,id Software,M,"XOne,PC",18 Wheels of Steel: Extreme Trucker,PC
11,Minecraft,2016,Misc,Microsoft Game Studios,0.28,0.17,0.18,0.04,0.68,77,73.0,93.0,4J Studios,E10+,WiiU,2002 FIFA World Cup,"XB,GC,PS2"
12,Mario & Sonic at the Rio 2016 Olympic Games,2016,Action,Nintendo,0.05,0.21,0.25,0.02,0.53,60,68.0,45.0,"Nintendo, Sega Sports R&D",E10+,"3DS,WiiU",2010 FIFA World Cup South Africa,"X360,PS3,PSP,Wii"
14,Hyrule Warriors,2016,Action,Nintendo,0.17,0.13,0.11,0.03,0.43,70,74.0,105.0,Omega Force,T,"3DS,WiiU",24: The Game,PS2
16,F1 2016 (Codemasters),2016,Racing,Codemasters,0.03,0.29,0.01,0.05,0.39,82,79.0,122.0,Codemasters,E,PS4,300: March to Glory,PSP
17,Resident Evil Zero,2016,Action,Capcom,0.10,0.17,0.05,0.05,0.37,70,77.0,183.0,Capcom,M,PS4,3D Dot Game Heroes,PS3


# Creating Dummies for non-numerical data (for columns : Type, Source, Rating)

In [507]:
# Created dummies for rating column

data_le_platform_dummy = data_le['Platform'].str.get_dummies(sep=',')
data_le_genre_dummy = data_le['Genre'].str.get_dummies(sep=',')
data_le_rating_dummy = data_le['Rating'].str.get_dummies(sep=',')
data_le_publisher_dummy = data_le['Publisher'].str.get_dummies(sep=',')


#merging all dummy dataframes to main dataframe

dataframes=[data_le,data_le_rating_dummy, data_le_platform_dummy,data_le_genre_dummy,data_le_publisher_dummy]

data_le_dummy =  reduce(lambda left,right: pd.merge(left, right, how='outer', left_index=True, right_index=True),  dataframes)
data_le_dummy = data_le_dummy.fillna(0)

In [796]:
data_le_dummy

,Name,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,User_Count,E,E10+,M,RP,T,3DS,DC,DS,GC,PC,PS,PS2,PS3,PS4,PSP,PSV,Wii,WiiU,X360,XB,XOne,Action,Adventure,Fighting,Misc,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy,Inc,10TACLE Studios,1C Company,2D Boy,3DO,505 Games,AQ Interactive,ASCII Entertainment,Acclaim Entertainment,Ackkstudios,Acquire,Activision,Activision Blizzard,Activision Value,Agatsuma Entertainment,Agetec,Aksys Games,Aqua Plus,Arc System Works,Ascaron Entertainment GmbH,Aspyr,Atari,Atlus,Avalon Interactive,BAM! Entertainment,Banpresto,Bethesda Softworks,Bigben Interactive,Black Bean Games,Blue Byte,Bohemia Interactive,Brash Entertainment,CCP,CDV Software Entertainment,Capcom,Cave,City Interactive,Cloud Imperium Games Corporation,Codemasters,Codemasters Online,Compile Heart,Conspiracy Entertainment,Crave Entertainment,Crimson Cow,D3Publisher,DHM Interactive,DSI Games,DTP Entertainment,Deep Silver,Destination Software,Destineer,Devolver Digital,Disney Interactive Studios,DreamCatcher Interactive,ESP,Eidos Interactive,Electronic Arts,Empire Interactive,Encore,Enix Corporation,Enterbrain,Evolved Games,Falcom Corporation,Flashpoint Games,Focus Home Interactive,Fox Interactive,From Software,FuRyu,Funbox Media,Funcom,Funsta,GOA,GT Interactive,Game Factory,Gamebridge,Gamecock,Gathering of Developers,Genki,Ghostlight,Global A Entertainment,Global Star,Gotham Games,Graffiti,Graphsim Entertainment,Groove Games,GungHo,Gust,HMH Interactive,Harmonix Music Systems,Hasbro Interactive,Havas Interactive,Hip Interactive,Home Entertainment Suppliers,Hudson Soft,Human Entertainment,Iceberg Interactive,Idea Factory,Idea Factory International,Ignition Entertainment,Indie Games,Infogrames,Interplay,Irem Software Engineering,Jaleco,Jester Interactive,JoWood Productions,Just Flight,Kadokawa Shoten,Kalypso Media,Kemco,Koch Media,Konami Digital Entertainment,Kool Kizz,Level 5,Lexicon Entertainment,Lighthouse Interactive,Little Orbit,LucasArts,MTV Games,Mad Catz,Majesco Entertainment,Marvelous Entertainment,Marvelous Interactive,Mastertronic,Mastiff,Max Five,Maxis,Media Rings,Mercury Games,Metro 3D,Microids,Microsoft Game Studios,Midas Interactive Entertainment,Midway Games,Milestone S.r.l.,Mindscape,Monte Christo Multimedia,Moss,Mud Duck Productions,Myelin Media,NCSoft,NDA Productions,Namco Bandai Games,Natsume,Navarre Corp,NewKidCo,Nihon Falcom Corporation,Nintendo,Nippon Ichi Software,Nobilis,Nordic Games,NovaLogic,Number None,O-Games,O3 Entertainment,Oxygen Interactive,PM Studios,PQube,Pacific Century Cyber Works,Paradox Interactive,Phantagram,Phantom EFX,Pinnacle,Play It,Playlogic Game Factory,PopCap Games,Popcorn Arcade,Psygnosis,RTL,Rage Software,Rebellion,RedOctane,Reef Entertainment,Revolution Software,Rising Star Games,Rondomedia,Russel,SCi,SNK,Sammy Corporation,Scholastic Inc.,Screenlife,Sega,Slightly Mad Studios,Sony Computer Entertainment,Sony Computer Entertainment America,Sony Computer Entertainment Europe,Sony Online Entertainment,SouthPeak Games,Spike,Square,Square EA,Square Enix,Square Enix,SquareSoft,Sting,Strategy First,Success,Sunflowers,Sunsoft,Swing! Entertainment,System 3,System 3 Arcade Software,TDK Mediactive,THQ,Taito,Takara Tomy,Take-Two Interactive,TalonSoft,Team17 Software,Tecmo Koei,Telegames,Telltale Games,Tetris Online,The Adventure Company,Titus,Tomy Corporation,Touchstone,Trion Worlds,Tripwire Interactive,Tru Blu Entertainment,Ubisoft,Ubisoft Annecy,Universal Interactive,Unknown,Valcon Games,Valve Software,Virgin Interactive,Visco,Vivendi Games,Wanadoo,Wargaming.net,Warner Bros. Interactive Entertainment,White Park Bay Software,XS Games,Xicat Interactive,Xplosiv,Xseed Games,Yacht Club Games,Zoo Digital Publishing,Zoo Games,Zushi Games,bitComposer Games,inXile Entertainment
0,Monster Hunter Stories,2017,0.00,0.00,0.28,0.00,0.28,79,82.0,91.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [508]:
data_le_dummy.Publisher.describe()

count                4278
unique                244
top       Electronic Arts
freq                  388
Name: Publisher, dtype: object

In [509]:
# data_le_dummy = data_le_dummy.reset_index()
data_le_dummy.reset_index()

,index,Name,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Score,User_Count,Developer,Rating,Platform_x,E,E10+,M,RP,T,3DS,DC,DS,GBA,GC,PC,PS,PS2,PS3,PS4,PSP,PSV,Wii,WiiU,X360,XB,XOne,Action,Adventure,Fighting,Misc,Platform_y,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy,Inc,10TACLE Studios,1C Company,2D Boy,3DO,505 Games,5pb,AQ Interactive,ASCII Entertainment,Acclaim Entertainment,Ackkstudios,Acquire,Activision,Activision Blizzard,Activision Value,Agatsuma Entertainment,Agetec,Aksys Games,Aqua Plus,Arc System Works,Ascaron Entertainment GmbH,Aspyr,Atari,Atlus,Avalon Interactive,BAM! Entertainment,Banpresto,Bethesda Softworks,Bigben Interactive,Black Bean Games,Blue Byte,Bohemia Interactive,Brash Entertainment,CCP,CDV Software Entertainment,Capcom,Cave,City Interactive,Cloud Imperium Games Corporation,Codemasters,Codemasters Online,Compile Heart,Conspiracy Entertainment,Crave Entertainment,Crimson Cow,D3Publisher,DHM Interactive,DSI Games,DTP Entertainment,Deep Silver,Destination Software,Destineer,Devolver Digital,Disney Interactive Studios,DreamCatcher Interactive,ESP,Eidos Interactive,Electronic Arts,Empire Interactive,Encore,Enix Corporation,Enterbrain,Evolved Games,Falcom Corporation,Flashpoint Games,Focus Home Interactive,Fox Interactive,From Software,FuRyu,Funbox Media,Funcom,Funsta,GOA,GT Interactive,Game Factory,Gamebridge,Gamecock,Gathering of Developers,Genki,Ghostlight,Global A Entertainment,Global Star,Gotham Games,Graffiti,Graphsim Entertainment,Groove Games,GungHo,Gust,HMH Interactive,Harmonix Music Systems,Hasbro Interactive,Havas Interactive,Hip Interactive,Home Entertainment Suppliers,Hudson Soft,Human Entertainment,Iceberg Interactive,Idea Factory,Idea Factory International,Ignition Entertainment,Indie Games,Infogrames,Interplay,Irem Software Engineering,Jaleco,Jester Interactive,JoWood Productions,Just Flight,Kadokawa Shoten,Kalypso Media,Kemco,Koch Media,Konami Digital Entertainment,Kool Kizz,Level 5,Lexicon Entertainment,Lighthouse Interactive,Little Orbit,LucasArts,MTV Games,Mad Catz,Majesco Entertainment,Marvelous Entertainment,Marvelous Interactive,Mastertronic,Mastiff,Max Five,Maxis,Media Rings,Mercury Games,Metro 3D,Microids,Microsoft Game Studios,Midas Interactive Entertainment,Midway Games,Milestone S.r.l.,Mindscape,Monte Christo Multimedia,Moss,Mud Duck Productions,Myelin Media,NCSoft,NDA Productions,Namco Bandai Games,Natsume,Navarre Corp,NewKidCo,Nihon Falcom Corporation,Nintendo,Nippon Ichi Software,Nobilis,Nordic Games,NovaLogic,Number None,O-Games,O3 Entertainment,Oxygen Interactive,PM Studios,PQube,Pacific Century Cyber Works,Paradox Interactive,Phantagram,Phantom EFX,Pinnacle,Play It,Playlogic Game Factory,PopCap Games,Popcorn Arcade,Psygnosis,RTL,Rage Software,Rebellion,RedOctane,Reef Entertainment,Revolution Software,Rising Star Games,Rondomedia,Russel,SCi,SNK,Sammy Corporation,Scholastic Inc.,Screenlife,Sega,Slightly Mad Studios,Sony Computer Entertainment,Sony Computer Entertainment America,Sony Computer Entertainment Europe,Sony Online Entertainment,SouthPeak Games,Spike,Square,Square EA,Square Enix,Square Enix,SquareSoft,Sting,Strategy First,Success,Sunflowers,Sunsoft,Swing! Entertainment,System 3,System 3 Arcade Software,TDK Mediactive,THQ,Taito,Takara Tomy,Take-Two Interactive,TalonSoft,Team17 Software,Tecmo Koei,Telegames,Telltale Games,Tetris Online,The Adventure Company,Titus,Tomy Corporation,Touchstone,Trion Worlds,Tripwire Interactive,Tru Blu Entertainment,Ubisoft,Ubisoft Annecy,Universal Interactive,Unknown,Valcon Games,Valve Software,Virgin Interactive,Visco,Vivendi Games,Wanadoo,Wargaming.net,Warner Bros. Interactive Entertainment,White Park Bay Software,XS Games,Xicat Interactive,Xplosiv,Xseed Games,Yacht Club Games,Zoo Digital Publishing,Zoo Games,Zushi Games,bitComposer Games,inXile Entertainment
0,0,Monster Hunter Stories,2017,Action,Capcom,0.00,0.00,0.28,0.00,0.28,79,82.0,91.0,Capcom,E10+,3DS,0,1,0,0,

In [510]:
# dropping columns for extra dummy variable and orginal column

data_le_dummy = data_le_dummy.drop(columns=['Platform_x','Genre','Publisher','Developer','Rating','GBA', 'Platform_y','Puzzle','5pb'])

In [512]:
data_le.describe(include = 'O').transpose()


,count,unique,top,freq
Name,4278,4278,Final Fantasy IV: The Complete Collection,1
Genre,4278,12,Action,839
Publisher,4278,244,Electronic Arts,388
Developer,4278,1180,Capcom,94
Rating,4278,5,T,1564
Platform,4278,402,PS2,597


# Exporting to CSV files

In [ ]:
# data_le.to_csv('data_le.csv', index = None, header=True)
# data_le_dummy.to_csv('data_le_dummy.csv', index = None, header=True)
# data_original.to_csv('data_original.csv', index = None, header=True)
